In [2]:
from sklearn.externals import joblib

joblib.load("output/AUDUSD_1540061539/DATA_PARAMS.pkl")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


{'raw_data_file': './DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv',
 'end_split': [datetime.datetime(2011, 1, 1, 0, 0),
  datetime.datetime(2013, 1, 1, 0, 0),
  datetime.datetime(2015, 1, 1, 0, 0),
  datetime.datetime(2018, 1, 1, 0, 0)],
 'TARGET_TO_PREDICT': 'AUDUSD',
 'FUTURE_PERIOD_PREDICT': 3,
 'TARGET_FUNCTION': <function cnn_preproc_function.cumulative_returns(returns)>,
 'SEQ_LEN': 60,
 'TARGET_THRESHOLD': 0.001,
 'FLIP': False}

In [6]:
from cnn_method import *
from cnn_preproc_function import *

import optparse
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing

import keras
import tensorflow as tf

from keras_model_configuration import *
from keras_metric import *

import datetime
import time
import os


# optparser = optparse.OptionParser()
# optparser.add_option("-a", "--assetindex", default=0, help="assetindex")
# optparser.add_option("-d", "--gpudevice", default=0, help="gpudevice")
# opts = optparser.parse_args()[0]

# assetindex = int(opts.assetindex)
# gpudevice = opts.gpudevice
assetindex = 0
gpudevice = "0"

os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]=gpudevice
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


possible_asset = ['AUDUSD', 'EURUSD', 'EURGBP', 'EURJPY', 'EWH', 'EWZ', 'FXI', 'IAU', 'EZU', 'KRE',
 'LQD', 'NZDUSD', 'EEM', 'EFA', 'GDX', 'HYG', 'SPY', 'XLU', 'IYR', 'GLD',
 'EWT', 'IEF', 'AGG', 'UNG', 'USDJPY', 'GBPUSD', 'USDCAD', 'SLV', 'RSX', 'EWJ',
 'OIH', 'SMH', 'XLB', 'TLT', 'USDCHF', 'USO', 'XLF', 'XLK', 'XLP', 'XOP',
 'VEA', 'VWO', 'XLE', 'XLI', 'XLV', 'XRT', 'XLY', 'VNQ', 'EWW', 'XBI']



if assetindex > len(possible_asset):
    raise Exception("assetindex > len")

print("=============================")
print(possible_asset[assetindex])
print("=============================")
DATA_PARAMS = dict()
DATA_PARAMS["raw_data_file"] = "./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv"
DATA_PARAMS["end_split"] = [datetime.datetime(2011,1,1), datetime.datetime(2013,1,1), datetime.datetime(2015,1,1), datetime.datetime(2018,1,1)]
DATA_PARAMS["TARGET_TO_PREDICT"] = possible_asset[assetindex]
DATA_PARAMS["FUTURE_PERIOD_PREDICT"] = 3
DATA_PARAMS["TARGET_FUNCTION"] = cumulative_returns
DATA_PARAMS["SEQ_LEN"] = 60
DATA_PARAMS["TARGET_THRESHOLD"] = 0.001
DATA_PARAMS["FLIP"] = False

MODEL_PARAMS = dict()
MODEL_PARAMS["BATCH_SIZE"] = 256
MODEL_PARAMS["EPOCHS"] = 200
MODEL_PARAMS["PATIENCE"] = 200
MODEL_PARAMS["LEARNING_RATE"] = 0.0005
MODEL_PARAMS["monitor_loss"] = "val_loss"
MODEL_PARAMS["mode_loss"] = "min"
INIT_TIME =  str(int(time.time()))
MODEL_PARAMS["project_folder"] = os.path.join("output", DATA_PARAMS["TARGET_TO_PREDICT"] + "_" + INIT_TIME)
MODEL_PARAMS["models_folder"] = os.path.join("output", DATA_PARAMS["TARGET_TO_PREDICT"] + "_" + INIT_TIME, "models")
MODEL_PARAMS["logs_folder"] = os.path.join("output", DATA_PARAMS["TARGET_TO_PREDICT"] + "_" + INIT_TIME, "logs")

print("Initialize Parameters: Done!")

locals().update(DATA_PARAMS)
locals().update(MODEL_PARAMS)

df = load_data(raw_data_file)
df = clean_and_create_target(df, TARGET_TO_PREDICT, FUTURE_PERIOD_PREDICT, TARGET_FUNCTION, TARGET_THRESHOLD, FLIP)
df, X, Y, start_index, end_index, scaler = split_df(df, end_split)
train_data_gen, val_data_gen, test_1_data_gen, test_2_data_gen, shape_x = TSGenerator(X, Y, SEQ_LEN, BATCH_SIZE, start_index, end_index)
class_weights = get_class_weights(df, start_index, end_index)

AUDUSD
Initialize Parameters: Done!
Load Data: Done!


/home/workstation/Desktop/CNN/cnn_preproc_function.py:27: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  df.loc[:,'target'] = df[target_col].rolling(window = FUTURE_PERIOD_PREDICT).apply(lambda x: FUNC(x))


Clean Data: Done!
(256, 60, 54)
Number of batches per epoch: 769
TSGenerator: Done!
[0.5808705  3.59136194]


In [10]:
df.columns

Index(['AUDUSD', 'EURUSD', 'EURGBP', 'EURJPY', 'EWH', 'EWZ', 'FXI', 'IAU',
       'EZU', 'KRE', 'LQD', 'NZDUSD', 'EEM', 'EFA', 'GDX', 'HYG', 'SPY', 'XLU',
       'IYR', 'GLD', 'EWT', 'IEF', 'AGG', 'UNG', 'USDJPY', 'GBPUSD', 'USDCAD',
       'SLV', 'RSX', 'EWJ', 'OIH', 'SMH', 'XLB', 'TLT', 'USDCHF', 'USO', 'XLF',
       'XLK', 'XLP', 'XOP', 'VEA', 'VWO', 'XLE', 'XLI', 'XLV', 'XRT', 'XLY',
       'VNQ', 'EWW', 'XBI', 'hh', 'mm', 'ss', 'wkday', 'target'],
      dtype='object')

In [24]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd

In [25]:
z = [j for j in range(100)]
z2 = [j+1 for j in range(100)]
Y = [j+0.1 for j in range(100)]
k = dict(z = z, z2 = z2, Y = Y)
df = pd.DataFrame(k)

In [46]:
p = TimeseriesGenerator(df[["z","z2"]].values, df["Y"].values,
                    length=5,
                    batch_size=10**9,
                    shuffle=False)

In [47]:
p[0]


(array([[[ 0.,  1.],
         [ 1.,  2.],
         [ 2.,  3.],
         [ 3.,  4.],
         [ 4.,  5.]],
 
        [[ 1.,  2.],
         [ 2.,  3.],
         [ 3.,  4.],
         [ 4.,  5.],
         [ 5.,  6.]],
 
        [[ 2.,  3.],
         [ 3.,  4.],
         [ 4.,  5.],
         [ 5.,  6.],
         [ 6.,  7.]],
 
        [[ 3.,  4.],
         [ 4.,  5.],
         [ 5.,  6.],
         [ 6.,  7.],
         [ 7.,  8.]],
 
        [[ 4.,  5.],
         [ 5.,  6.],
         [ 6.,  7.],
         [ 7.,  8.],
         [ 8.,  9.]],
 
        [[ 5.,  6.],
         [ 6.,  7.],
         [ 7.,  8.],
         [ 8.,  9.],
         [ 9., 10.]],
 
        [[ 6.,  7.],
         [ 7.,  8.],
         [ 8.,  9.],
         [ 9., 10.],
         [10., 11.]],
 
        [[ 7.,  8.],
         [ 8.,  9.],
         [ 9., 10.],
         [10., 11.],
         [11., 12.]],
 
        [[ 8.,  9.],
         [ 9., 10.],
         [10., 11.],
         [11., 12.],
         [12., 13.]],
 
        [[ 9., 10.],
       

In [50]:
df["z"][5:]

5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
24    24
25    25
26    26
27    27
28    28
29    29
30    30
31    31
32    32
33    33
34    34
      ..
70    70
71    71
72    72
73    73
74    74
75    75
76    76
77    77
78    78
79    79
80    80
81    81
82    82
83    83
84    84
85    85
86    86
87    87
88    88
89    89
90    90
91    91
92    92
93    93
94    94
95    95
96    96
97    97
98    98
99    99
Name: z, Length: 95, dtype: int64